In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import skimage
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
%matplotlib inline

In [ ]:
data = pd.read_csv(r'../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
col = ['r','c']
sns.countplot(x=data['gender'], palette=col)

In [ ]:
col=['c','g']
sns.countplot(x=data['stroke'], palette=col)

In [ ]:
cmap = sns.diverging_palette(300,150,s=50,l=75,n=9)
corrmat = data.corr()
plt.subplots(figsize=(18,18))
sns.heatmap(corrmat, cmap=cmap, annot=True, square=True)

In [ ]:
col=['c','g']
sns.countplot(x=data['ever_married'], palette=col)

In [ ]:
col=['c','g','b','r']
sns.countplot(x=data['smoking_status'], palette=col)

In [ ]:
col=['c','g','r','b']
sns.countplot(x=data['work_type'], palette=col)

In [ ]:
col=['c','g']
sns.countplot(x=data['Residence_type'], palette=col)

In [ ]:
from sklearn.preprocessing import LabelEncoder 
  
le = LabelEncoder() 
  
data['gender']= le.fit_transform(data['gender'])
data['Residence_type']=le.fit_transform(data['Residence_type'])
data['ever_married']= le.fit_transform(data['ever_married'])


In [ ]:
data.head()

## data to be use for prediction and training are :- age, gender, heart_disease, avg_glucose, bmi, smoking_status

In [ ]:
data.tail(10)

In [ ]:
plt.figure(figsize=(18,18))
days = sns.countplot(x=data['smoking_status'],data=data,hue='stroke',palette=col)
days.set_title('stroke per week')

In [ ]:
plt.figure(figsize=(28,18))
days = sns.countplot(x=data['age'],data=data,hue='stroke',palette=col)
days.set_title('stroke per week')

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data.work_type = le.fit_transform(data.work_type)
data.smoking_status = le.fit_transform(data.smoking_status)
data.head()

# Converting smoking_type into binary variables

In [ ]:
data = data.drop(['id','work_type','Residence_type'],axis=1)

In [ ]:
print(type(data))

In [ ]:
data.head()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
ct = ColumnTransformer(transformers=[('ohe',OneHotEncoder(),[7])],remainder='passthrough')

In [ ]:
data2=ct.fit_transform(data)

In [ ]:
data2

In [ ]:
data2=pd.DataFrame(data2)

In [ ]:
data2

In [ ]:
import statistics
x = statistics.median(data2[10])

In [ ]:
x

In [ ]:
data2 = data2.fillna(x)

In [ ]:
data2.head(10)

In [ ]:
X = data2.drop(11,axis=1)

In [ ]:
print(X)

In [ ]:
y = data2[11]

In [ ]:
y

In [ ]:
X = pd.DataFrame(X)

In [ ]:
X

In [ ]:
y = pd.DataFrame(y)
y

In [ ]:
 from sklearn.preprocessing import StandardScaler

In [ ]:
s_scaler = StandardScaler()

In [ ]:
X = pd.DataFrame(X)
X.head()

In [ ]:
data.head()

In [ ]:
X.head()

In [ ]:
X = s_scaler.fit_transform(X)

In [ ]:
X = pd.DataFrame(X)

In [ ]:
X  = s_scaler.fit_transform(X)
X = pd.DataFrame(X)

In [ ]:
X.describe().T

In [ ]:
X.info()

In [ ]:
y.info()

In [ ]:
colors = ['#774571','#b398af','#f1f1f1','#afada7','#6daa9f']
plt.figure(figsize=(20,10))
sns.boxenplot(data=X,palette=colors)
plt.xticks(rotation=90)
plt.show()

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
X_train

In [ ]:
y_train

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
X

In [ ]:
y

## Model Building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.metrics import binary_accuracy
model = Sequential()

In [ ]:
model.add(Dense(units=11,kernel_initializer='uniform',activation='relu',input_dim=11))
model.add(Dense(units=11,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=9,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=7,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=5,kernel_initializer='uniform',activation='relu'))
model.add(Dense(units=1,kernel_initializer='uniform',activation='relu'))


## Compiling

In [ ]:
model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

## Train ANN model

In [ ]:
history = model.fit(X_train, y_train, batch_size=10,epochs=250,validation_split=0.2)

In [ ]:
val_accuracy = np.mean(history.history['val_accuracy'])
print('\n%s : %.2f%%' %('val_Accuracy',val_accuracy*100))

## Conlusion

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
np.set_printoptions()

In [ ]:
from sklearn.metrics import confusion_matrix,precision_score,recall_score,classification_report,accuracy_score,f1_score
#cmap1 = sns.diverging_palette(275,150,s=40,l=65,n=6)
#plt.subplots(figsize=(12,8))
cf_matrix = confusion_matrix(y_test,y_pred)
#sns.heatmap(cf_matrix/np.sum(cf_matrix),cmap=cmap1,annot=True,annot_kws={'size':15})
print(classification_report(y_test,y_pred))

In [ ]:
pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv').to_csv('my_output.csv')